<a href="https://colab.research.google.com/github/ArshockAbedan/ML2021/blob/main/NLP_RNN_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

# import os
# os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"


tfds.disable_progress_bar()

In [ ]:
tf.__version__

'2.8.0'

In [ ]:
dataset, info = tfds.load('imdb_reviews', data_dir='.', with_info=True, as_supervised=True)

Shuffling and writing examples to ./imdb_reviews/plain_text/1.0.0.incompleteY6KRRR/imdb_reviews-train.tfrecord
Shuffling and writing examples to ./imdb_reviews/plain_text/1.0.0.incompleteY6KRRR/imdb_reviews-test.tfrecord
Shuffling and writing examples to ./imdb_reviews/plain_text/1.0.0.incompleteY6KRRR/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to ./imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [ ]:
dataset

{'test': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'train': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'unsupervised': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [ ]:
train_dataset, test_dataset = dataset['train'], dataset['test']

In [ ]:
type(train_dataset)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [ ]:
len(train_dataset)

25000

In [ ]:
len(test_dataset)

25000

In [ ]:
for sample in train_dataset:
    print(sample[0].numpy())
    print('\n')
    print(sample[1].numpy())
    break

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."


0


In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
for example, label in train_dataset.take(1):
    print('texts: ', example.numpy()[:3])
    print('\n')
    print('labels: ', label.numpy()[:3])

texts:  [b'***SPOILERS*** ***SPOILERS*** When I saw a preview for this movie I thought it was going to be atleast a slightly admirable storyline. But as most superstation original movies I was left disapointed. This gullible family ends up driving through this "deserted" town to take a brake and find this video camera showing these people doing everything their donig and finds out they all eventually disappear, the family goes through all these mysterious stages and never discovers or displays what the heck is stalking them. Their are more gaps than I can count and they don\'t explain anything that happens how or what. It ends where the family gets in a car accident and get posest or brainwashed or something( which is never explained). The next thing you know ur hoping they somehow find out how all it happened but it ends leaving you completely confused.'
 b"Scott Menville is not Casey Kasem. That is the first, most important, and most disturbing thing about this attempt at re-imaginin

In [ ]:
e = tf.keras.layers.experimental.preprocessing.TextVectorization()
e.adapt([
    "I love samosas and jalebi",
    "I love biking and yoga",
    "I love tensorflow"
])

In [ ]:
e.get_vocabulary()

['',
 '[UNK]',
 'love',
 'i',
 'and',
 'yoga',
 'tensorflow',
 'samosas',
 'jalebi',
 'biking']

In [ ]:
e(["I love pizza"]).numpy()

array([[3, 2, 1]])

In [ ]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:25]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but',
       'film', 'on', 'not', 'you', 'are'], dtype='<U14')

In [ ]:
example[:2]

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'***SPOILERS*** ***SPOILERS*** When I saw a preview for this movie I thought it was going to be atleast a slightly admirable storyline. But as most superstation original movies I was left disapointed. This gullible family ends up driving through this "deserted" town to take a brake and find this video camera showing these people doing everything their donig and finds out they all eventually disappear, the family goes through all these mysterious stages and never discovers or displays what the heck is stalking them. Their are more gaps than I can count and they don\'t explain anything that happens how or what. It ends where the family gets in a car accident and get posest or brainwashed or something( which is never explained). The next thing you know ur hoping they somehow find out how all it happened but it ends leaving you completely confused.',
       b"Scott Menville is not Casey Kasem. That is the first, most important, and most 

In [ ]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[  1,   1,  51, ...,   0,   0,   0],
       [  1,   1,   7, ...,   0,   0,   0],
       [ 10, 373,  11, ...,   0,   0,   0]])

In [ ]:
for n in range(3):
    print("Original: ", example[n].numpy())
    print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
    print('\n')

Original:  b'***SPOILERS*** ***SPOILERS*** When I saw a preview for this movie I thought it was going to be atleast a slightly admirable storyline. But as most superstation original movies I was left disapointed. This gullible family ends up driving through this "deserted" town to take a brake and find this video camera showing these people doing everything their donig and finds out they all eventually disappear, the family goes through all these mysterious stages and never discovers or displays what the heck is stalking them. Their are more gaps than I can count and they don\'t explain anything that happens how or what. It ends where the family gets in a car accident and get posest or brainwashed or something( which is never explained). The next thing you know ur hoping they somehow find out how all it happened but it ends leaving you completely confused.'
Round-trip:  [UNK] [UNK] when i saw a [UNK] for this movie i thought it was going to be [UNK] a [UNK] [UNK] storyline but as most 

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
sample_text = ('awesome movie, I loved it so much')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.00274829]


In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [25]:
model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
391/391 [==============================] - 107s 244ms/step - loss: 0.6562 - accuracy: 0.5591 - val_loss: 0.5358 - val_accuracy: 0.7089
Epoch 2/10
391/391 [==============================] - 92s 234ms/step - loss: 0.4252 - accuracy: 0.8018 - val_loss: 0.3771 - val_accuracy: 0.8359
Epoch 3/10
391/391 [==============================] - 93s 236ms/step - loss: 0.3536 - accuracy: 0.8441 - val_loss: 0.3454 - val_accuracy: 0.8510
Epoch 4/10
391/391 [==============================] - 93s 235ms/step - loss: 0.3264 - accuracy: 0.8582 - val_loss: 0.3293 - val_accuracy: 0.8516
Epoch 5/10
391/391 [==============================] - 93s 236ms/step - loss: 0.3164 - accuracy: 0.8621 - val_loss: 0.3274 - val_accuracy: 0.8609
Epoch 6/10
391/391 [==============================] - 91s 231ms/step - loss: 0.3098 - accuracy: 0.8656 - val_loss: 0.3273 - val_accuracy: 0.8474
Epoch 7/10
391/391 [==============================] - 92s 232ms/step - loss: 0.3039 - accuracy: 0.8693 - val_loss: 0.3190 - val_a

**Check Sample Text Again - After Training**

In [26]:
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[1.2167441]


Now it is not near to zero and it is not Negetive at all.

In [27]:
sample_text_for_test = ('Misery and Stand By Me were the best adaptations up until this one, now you can add Shawshank to that list.')


predictions = model.predict(np.array([sample_text_for_test]))
print(predictions[0])

[1.8862258]
